# Note
Change `file_path` to your file location where `articles_metadata.csv` is at.

In [1]:
import os
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
def get_article_text(link, file_path):
    """Fetches article text, handles variations, and saves to file."""
    retries = 3
    for attempt in range(retries):
        driver = None
        try:
            driver = webdriver.Chrome()
            driver.get(link)

            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '._article_content')))
            except:
                print(f"Timed out waiting for content on {link} (attempt {attempt+1})")
                continue

            soup = BeautifulSoup(driver.page_source, "html.parser")
            content_div = soup.select_one("._article_content")

            if not content_div:
                print(f"Could not find _article_content div on {link} (attempt: {attempt+1})")
                continue

            article_text = content_div.get_text(strip=True, separator='\n')

            if article_text:
                directory = os.path.dirname(file_path)
                
                # If a directory path exists and the folder is missing, create it
                if directory and not os.path.exists(directory):
                    os.makedirs(directory, exist_ok=True)

                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(article_text)
                return
            else:
                print(f"No article text found for {link} (attempt: {attempt+1})")

        except Exception as e:
            print(f"Error processing {link} (attempt: {attempt+1}) {e}")
            time.sleep(5)

        finally:
            if driver:
                driver.quit()

    print(f"Failed to process {link} after {retries} retries.")

In [6]:
def main():
    file_path = "C:/Users/WINDOWS 11/Desktop/kpop_agenda/data/articles_metadata.csv"
    article_data = []
    try:  
        with open(file_path, "r", encoding="utf-8") as csvfile:
            reader = csv.DictReader(csvfile) 
            for row in reader:
                article_data.append((row["link"], row["file_path"]))
    except FileNotFoundError:
        print(f"Error: CSV file not found at {file_path}")
        return  
    except Exception as e: 
        print(f"Error reading CSV file: {e}")
        return

    with ThreadPoolExecutor(max_workers=8) as executor:  
        futures = {executor.submit(get_article_text, link, file_path): link for link, file_path in article_data}

        for future in as_completed(futures):
            link = futures[future]
            if future.exception() is not None:
                print(f"An error occurred while processing {link}: {future.exception()}")
            else:
                print(f"Finished processing {link}")

if __name__ == "__main__":
    main()

Finished processing https://m.entertain.naver.com/ranking/article/076/0004221884
Finished processing https://m.entertain.naver.com/ranking/article/609/0000927624
Finished processing https://m.entertain.naver.com/ranking/article/312/0000691155
Finished processing https://m.entertain.naver.com/ranking/article/144/0001004352
Finished processing https://m.entertain.naver.com/ranking/article/108/0003286719
Finished processing https://m.entertain.naver.com/ranking/article/109/0005205388
Finished processing https://m.entertain.naver.com/ranking/article/052/0002121494
Finished processing https://m.entertain.naver.com/ranking/article/609/0000927432
Finished processing https://m.entertain.naver.com/ranking/article/311/0001802134
Finished processing https://m.entertain.naver.com/ranking/article/076/0004221839
Finished processing https://m.entertain.naver.com/ranking/article/477/0000524174
Finished processing https://m.entertain.naver.com/ranking/article/108/0003286833
Finished processing https://

# Testing part below, retriving one article's full text to test.
You can change `test_url` to any news articles in Naver entertainment.

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_article_text(link):
    try:
        driver = webdriver.Chrome()
        driver.get(link)

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '._article_content')))

        soup = BeautifulSoup(driver.page_source, "html.parser")
        print(driver.title)

        content_div = soup.select_one("._article_content")
        if content_div:
            all_text = content_div.get_text(strip=True, separator='\n')
        else:
            print("Could not find _article_content div.")

    except Exception as e:
        print(f"Error: {e}")
    finally:
        if driver:
            driver.quit()

test_url = "https://m.entertain.naver.com/ranking/article/311/0001677035"
get_article_text(test_url)

박민영, 열애설 논란 언급 "매일 후회해…다신 불미스러운 일 없게 할 것"
(엑스포츠뉴스 오승현 기자) 박민영이 힘들었던 지난 해 개인사를 언급했다.
1일 오전, tvN 새 월화드라마 '내 남편과 결혼해줘'의 온라인 제작발표회가 진행됐다. 이 자리에는 박원국 감독을 비롯해 배우 박민영, 나인우, 이이경, 송하윤, 이기광이 참석했다.
'내 남편과 결혼해줘'는 절친과 남편의 불륜을 목격하고 살해당한 여자가 10년 전으로 회귀해 인생 2회차를 경험하며 시궁창 같은 운명을 그들에게 돌려주는 본격 운명 개척 드라마다.
박민영은 열애설이 났던 전 남자친구 강종현의 논란과 관련해 함께 언급이 되어 왔다. 두 사람은 이후 관계를 정리했고, 박민영은 논란 후 첫 작품으로 '내 남편과 결혼해줘'를 택했다.
개인사로 힘들었던 2023년을 회상한 박민영은 "사실 몸도 정신 건강도 아팠던 해다. 과연 내가 잘할 수 있을까 생각하던 차에 감독님이 잘 잡아주셨다"라며 배우로 돌아온 소감을 전했다.
이어 박민영은 "저도 짧다면 짧은 시간이지만 정말 매일 후회하고 지냈었다. 정신과에서 뇌파 검사를 했을 때도 죄책감에 위험 신호가 떴다. 그 시간들이 오히려 제게 본업이 무엇이고 언제 행복한지, 촬영장에서 예전처럼 연기만 할 때가 예쁘고 빛난다는 깨달음을 줬다"고 당시를 솔직히 이야기했다.
그는 "지금 모든 걸 받아들였고 심려 끼친 것에 대해 꼭 말씀드리고 싶었다. 더 일찍이라면 좋았겠지만 제가 지금 건강해진 상태라 진정성 있게 (하고 싶었다)"고 밝혔다.
박민영은 "절 사랑해주신 분, 아껴주신 분들, 팬분들에게 가장 죄송하다"며 "다시는 다른 이슈로 불미스러운 일 일어나지 않게 제가 배우로서 정말 좋은 모습 보여드리는 게 답인 것 같다"며 배우로서의 굳건한 모습을 약속했다.
그는 "항상 자랑스러운 배우가 되겠다고 다짐했는데 후회하고 있다. 다시 돌아가려고 하고 있다. 감사하다"고 덧붙여 제작발표회를 시청하던 네티즌들의 응원을 받았다.
한편, '내 남편과 결혼해줘'는 1일 오후 8시 50분에 